This notebook preprocesses the cell towers dataset into an efficient set of parquet files. First register for an API key with [opencellid](https://www.opencellid.org/) and then download the `cell_towers.csv.gz` file.  Unzip it and modify the `cell_towers_path` below to the path to your `cell_towers.csv` file.  Then execute this notebook to write out a compressed parquet to the `../data` directory.

In [1]:
cell_towers_path = "assets/505.csv"


In [2]:
# Imports
import dask.dataframe as dd
import pandas as pd
import pyproj
from pyproj import Transformer
import requests
import os

In [3]:
# From https://www.opencellid.org/downloads.php
# Field Descriptions: http://wiki.opencellid.org/wiki/Menu_map_view#Cells_database:
ddf = dd.read_csv(cell_towers_path)
ddf.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal
0,UMTS,505,1,49232,13918075,0,152.901143,-31.432680,1000,277,1,1290219324,1290219324,0
1,UMTS,505,1,49232,13918073,0,152.901665,-31.433336,1000,106,1,1290219193,1290219193,0
2,UMTS,505,1,49232,13901334,0,152.901535,-31.433945,1000,2,1,1290126925,1290126925,0
3,UMTS,505,1,49232,13901338,0,152.901428,-31.433251,1000,103,1,1290274256,1290823466,0
4,UMTS,505,1,49232,13901335,0,152.899138,-31.430498,1000,51,1,1290205885,1290205885,0


In [4]:
# Categorize radio
ddf['radio'] = ddf.radio.astype('category')

# Created and updated to datetime integers
ddf['created'] = dd.to_datetime(ddf.created, unit='s')
ddf['updated'] = dd.to_datetime(ddf.updated, unit='s')

# Filter out outliers created before 2003
ddf = ddf[dd.to_datetime(ddf.created) >= '2003']

In [5]:
# convert lon/lat to epsg:3857 (psuedo-mercator) so generated images
# can be overlayed on a Mercator projected map
transformer = Transformer.from_crs("epsg:4326", "epsg:3857")
def to3857(df):
    x_3857, y_3857 = transformer.transform(df.lat.values, df.lon.values)
    return df.assign(x_3857=x_3857, y_3857=y_3857)

ddf = ddf.map_partitions(to3857)

ddf.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal,x_3857,y_3857
0,UMTS,505,1,49232,13918075,0,152.901143,-31.432680,1000,277,1,2010-11-20 02:15:24,2010-11-20 02:15:24,0,1.702088e+07,-3.689069e+06
1,UMTS,505,1,49232,13918073,0,152.901665,-31.433336,1000,106,1,2010-11-20 02:13:13,2010-11-20 02:13:13,0,1.702094e+07,-3.689155e+06
2,UMTS,505,1,49232,13901334,0,152.901535,-31.433945,1000,2,1,2010-11-19 00:35:25,2010-11-19 00:35:25,0,1.702092e+07,-3.689234e+06
3,UMTS,505,1,49232,13901338,0,152.901428,-31.433251,1000,103,1,2010-11-20 17:30:56,2010-11-27 02:04:26,0,1.702091e+07,-3.689144e+06
4,UMTS,505,1,49232,13901335,0,152.899138,-31.430498,1000,51,1,2010-11-19 22:31:25,2010-11-19 22:31:25,0,1.702065e+07,-3.688785e+06


In [6]:
# Download network info for mcc/mnc from 'https://cellidfinder.com/mcc-mnc'
html = requests.get('https://cellidfinder.com/mcc-mnc')
tables = pd.read_html(html.content)
mcc_mnc_df = pd.concat(tables).reset_index(drop=True)

# Create description column as Network, falling back to "Operator or branch" if Network not found
mcc_mnc_df['Description'] = mcc_mnc_df.Network.where(
    ~pd.isnull(mcc_mnc_df.Network), mcc_mnc_df['Operator or brand name']
)

# Drop unneeded columns
codes = mcc_mnc_df.drop(['Network', 'Operator or brand name'], axis=1)
codes.head()

,MCC,MNC,Status,Description
0,289,67,Operational,Aquafon
1,289,68,Operational,A-Mobile
2,412,1,Operational,Afghan Wireless Communication Company
3,412,20,Operational,Telecom Development Company Afghanistan Ltd.
4,412,40,Operational,MNT Group Afganistan


In [7]:
# Categorize non-numeric columns
for col, dtype in codes.dtypes.items():
    if dtype == 'object':
        codes[col] = codes[col].astype('category')

In [8]:
# Merge mnc/mcc info with cell towers dataset
ddf_merged = ddf.merge(codes, left_on=['mcc', 'net'], right_on=['MCC', 'MNC'], how='left')
ddf_merged

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal,x_3857,y_3857,MCC,MNC,Status,Description
npartitions=1,,,,,,,,,,,,,,,,,,,,
,category[unknown],int64,int64,int64,int64,int64,float64,float64,int64,int64,int64,datetime64[ns],datetime64[ns],int64,float64,float64,int64,int64,category[known],category[known]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
# Write parquet file to ../data directory
os.makedirs('assets/', exist_ok=True)
parquet_path = 'assets/cell_towers.parq'
ddf_merged.to_parquet(parquet_path, compression='snappy')

In [10]:
# Read and display the first three rows (Transpose so can see all of the columns)
dd.read_parquet(parquet_path).head(3).T

,0,1,2
radio,UMTS,UMTS,UMTS
mcc,505,505,505
net,1,1,1
area,49232,49232,49232
cell,13918075,13918073,13901334
unit,0,0,0
lon,152.901143,152.901665,152.901535
lat,-31.43268,-31.433336,-31.433945
range,1000,1000,1000
samples,277,106,2
